In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

# Project Source: https://github.com/thu-vu92/python-dashboard-panel/blob/main/Interactive_dashboard.ipynb

In [2]:
# df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")

In [3]:
# improve dashboard performance by caching data
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")
    pn.state.cache['data'] = df.copy()
else:
    df = pn.state.cache['data']

In [4]:
print(df.head())
df.columns

       country  year iso_code  population  gdp  cement_co2  \
0  Afghanistan  1850      AFG   3752993.0  NaN         NaN   
1  Afghanistan  1851      AFG   3769828.0  NaN         NaN   
2  Afghanistan  1852      AFG   3787706.0  NaN         NaN   
3  Afghanistan  1853      AFG   3806634.0  NaN         NaN   
4  Afghanistan  1854      AFG   3825655.0  NaN         NaN   

   cement_co2_per_capita  co2  co2_growth_abs  co2_growth_prct  ...  \
0                    NaN  NaN             NaN              NaN  ...   
1                    NaN  NaN             NaN              NaN  ...   
2                    NaN  NaN             NaN              NaN  ...   
3                    NaN  NaN             NaN              NaN  ...   
4                    NaN  NaN             NaN              NaN  ...   

   share_global_cumulative_other_co2  share_global_flaring_co2  \
0                                NaN                       NaN   
1                                NaN                       NaN   
2 

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [5]:
# fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df["gdp_per_capita"] = np.where(df['population'] != 0, df['gdp']/ df['population'], 0)

In [6]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

h4 (1) CO2 emissions over time by continent

In [7]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1750, end=2020, step=5, value=2020)
year_slider

IntSlider(end=2020, name='Year slider', start=1750, step=5, value=2020)

In [8]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['co2', 'co2_per_capita',],
    button_type='success'
)

In [9]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [10]:
co2_pipeline

In [11]:
co2_plot = co2_pipeline.hvplot(x = 'year', by = 'country', y = yaxis_co2, line_width = 2, title = "CO2 emission by continent")
co2_plot

h4 (2) Table - CO2 emission over time by continent

In [12]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode = 'stretch_width')
co2_table

h4 (3) CO2 vs GDP scatterplot

In [13]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        ( ~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [14]:
co2_vs_gdp_scatterplot_pipeline

In [15]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', y='co2', by='country', size=80, kind = "scatter", alpha=0.7, legend=False, height=500, width=500)
co2_vs_gdp_scatterplot

(4) Bar chart with CO2 sources by continent

In [16]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

continents_excl_world = ['Aisa', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [17]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind = 'bar', x = 'country', y = yaxis_co2_source, title = 'CO2 source by continent')
co2_source_bar_plot

Creating Dashboard

In [20]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard',
    sidebar=[pn.pane.Markdown('# CO2 Emissions and Climate Change'),
            pn.pane.Markdown("#### Carbon dioxide emissons are the primary driver of global climate change. It's widely recognised that to avoid the worst effects of climate change, the maximum temperature rise should kept to a maximum of 2°C."),
            pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
            pn.pane.Markdown('## Settings'),
            year_slider],
    main=[pn.Row(pn.Column(yaxis_co2,
                          co2_plot.panel(width=700), margin=(0.25)),
                co2_table.panel(width=500)),
         pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0.25)),
               pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0"
)
template.show()
template.servable();

Launching server at http://localhost:50846
